# OUTDATED
look at version without 't2'

# Imports  

In [1]:
import os, sys
import pandas as pd
import numpy as np
import pickle, json
import process_redcap 
from scipy import signal
import os.path



## Get RedCap

In [2]:
radwear_path = '/projects/bbnp/Vansh/data/RADWear/'
redcap_path = radwear_path+'REDCap responses/'

# load all participant meta data
with open(radwear_path+'all_p_metadata.json', 'rb') as f:
            all_p_metadata = json.load(f)
# load all participant redcap data
redcap_df = process_redcap.process_redcap(redcap_path,all_p_metadata['list of participant IDs'])
redcap_dfcalib = redcap_df.copy() # a cheat for now since i don't have redcap data for calibration
columns_list = redcap_dfcalib.columns

###################
# these are temporary fixes for the redcap data. if main redcap data is updated, these fixes should be removed
# #fixed for p7 
#redcap_df.at[28, 'daily_check_in_date'] = '2022-09-12'
#redcap_df.at[30, 'daily_check_in_date'] = '2022-09-13'
#redcap_df.iloc[29], redcap_df.iloc[30] =  redcap_df.iloc[30].copy(), redcap_df.iloc[29].copy()
#redcap_df.drop([44], inplace = True)

redcap dict exists
redcap df exists
redcap df pickle loaded


# get metadata

In [3]:
radwear_path = '/projects/bbnp/Vansh/data/RADWear/'
# load the data

#import all_p_metadata.json
with open(radwear_path + 'all_p_metadata.json') as f:
    all_p_metadata = json.load(f)

list_of_participants = all_p_metadata['list of participant IDs'].copy()
print('original list of participants: ',list_of_participants)
list_of_participants.remove(4) # remote calibration only participants for now until i add the redcap data for them
list_of_participants.remove(18) # remote calibration only participants for now until i add the redcap data for them


print('list of participants to process: ', list_of_participants)
print('-----------------------')
fs = all_p_metadata['fs']

rot_anx_dict = {'calibration': 0, 'LA': 1, 'HA': 2}

original list of participants:  [4, 7, 9, 12, 14, 16, 17, 18, 21]
list of participants to process:  [7, 9, 12, 14, 16, 17, 21]
-----------------------


In [4]:
p = 7
p_path = radwear_path + 'Participant '+str(p)
print(p_path)

/projects/bbnp/Vansh/data/RADWear/Participant 7


In [29]:
list_of_participants[4:]

[16, 17, 21]

# load each P data and process

In [30]:
#all_p = {}
#load each participant's data
force_update = True
for p in list_of_participants[4:]:
    p_path = radwear_path + 'Participant '+str(p)
    p_df = pd.DataFrame()
    
    #check if file exist
    a = 'available.' if os.path.isfile(p_path + '/p_'+str(p)+'.pkl') else ' not available.'
    print('pickle file for participant '+str(p)+ ' is '+ a)
    with open(p_path +  '/p_'+str(p)+'.pkl', 'rb') as f:
        p_data = pickle.load(f)
    print ('participant '+str(p)+ ' data loaded.')
    print('-----------------------') 
    #all_p[p] = p_data ## this takes too much memory so i will just load each p when needed
    if not (os.path.isfile(radwear_path +'p_'+str(p)+'.pkl')) or force_update:

        print('Participant: ', p)
        print('-----------------------')    
        print('p_data.keys(): ', p_data.keys())
        print('-----------------------')
        print('la days: ', len(p_data['HA'].keys()))
        print('ha days: ', len(p_data['LA'].keys()))
        print('-----------------------')
        #print('p_data[calib].keys(): ', p_data['calib'].keys())
        # e4 contains BVP, EDA, TEMP, ACC, IBI, HR, HRV, tags
        # hx contains ECG, ACC, BR
        p_redcap = redcap_df.loc[redcap_df['participant'] == p] # redcap data for participant p


        ## this is for participant p 

        for day in list(p_data['LA'].keys()): 

            proceed1 = bool(len(p_data['LA'][day])) and bool(all_p_metadata[str(p)]['RedCap available'][0][day])
            
            if proceed1:
                
                LA_df = p_data['LA'][day]['BVP'].copy()
                del p_data['LA'][day]['BVP']
                resample_ratio = round(len(LA_df))

                LA_df['ECG'] = signal.resample(p_data['LA'][day]['ECG']['ECG'], resample_ratio)
                del p_data['LA'][day]['ECG']
                #LA_df['BVP'] = signal.resample(p_data['LA'][day]['BVP']['BVP'], resample_ratio)
                LA_df['EDA'] = signal.resample(p_data['LA'][day]['EDA']['EDA'], resample_ratio)
                del p_data['LA'][day]['EDA']
                LA_df['TEMP'] = signal.resample(p_data['LA'][day]['TEMP']['Temp'], resample_ratio)
                del p_data['LA'][day]['TEMP']
                LA_df['BR'] = signal.resample(p_data['LA'][day]['BR']['breathing_rate'], resample_ratio)
                del p_data['LA'][day]['BR']
                LA_df['ACCx_hx'] = signal.resample(p_data['LA'][day]['ACC_hx']['Acceleration_X'], resample_ratio)
                del p_data['LA'][day]['ACC_hx']['Acceleration_X']
                LA_df['ACCy_hx'] = signal.resample(p_data['LA'][day]['ACC_hx']['Acceleration_Y'], resample_ratio)
                del p_data['LA'][day]['ACC_hx']['Acceleration_Y']
                LA_df['ACCz_hx'] = signal.resample(p_data['LA'][day]['ACC_hx']['Acceleration_Z'], resample_ratio)
                del p_data['LA'][day]['ACC_hx']['Acceleration_Z']   
                LA_df['ACCx_e4'] = signal.resample(p_data['LA'][day]['ACC_e4']['Acc_X'], resample_ratio)
                del p_data['LA'][day]['ACC_e4']['Acc_X']
                LA_df['ACCy_e4'] = signal.resample(p_data['LA'][day]['ACC_e4']['Acc_Y'], resample_ratio)
                del p_data['LA'][day]['ACC_e4']['Acc_Y']
                LA_df['ACCz_e4'] = signal.resample(p_data['LA'][day]['ACC_e4']['Acc_Z'], resample_ratio)
                del p_data['LA'][day]['ACC_e4']['Acc_Z']
                LA_df['rot_label'] = rot_anx_dict['LA']* np.ones(resample_ratio)
                LA_df['calib_label'] = np.zeros(resample_ratio)

                for column in columns_list:
                    if column == 'daily_check_in_date':
                        A = np.chararray(resample_ratio,itemsize=15,unicode=True)
                        A[:] = ((redcap_df.loc[redcap_df['participant'] == p]).iloc[day][column])
                        LA_df[column] = A
                    else:
                        LA_df[column] = np.ones(resample_ratio)* (redcap_df.loc[redcap_df['participant'] == p]).iloc[day][column] 
                
                print('Done with day ', day, 'date: ',(redcap_df.loc[redcap_df['participant'] == p]).iloc[day]['daily_check_in_date'],' of LA rotation for participant ', p)               
                p_df = pd.concat([p_df, LA_df], ignore_index=True, sort=False)
                #print('display LA_df after LA_df')
                #display(LA_df)
                

        for day in list(p_data['HA'].keys()):
            proceed2 = bool(len(p_data['HA'][day])) and bool(all_p_metadata[str(p)]['RedCap available'][1][day])
            if proceed2:
                    
                
                HA_df = p_data['HA'][day]['BVP'].copy()
                del p_data['HA'][day]['BVP']
                resample_ratio = round(len(HA_df))
                HA_df['ECG'] = signal.resample(p_data['HA'][day]['ECG']['ECG'], resample_ratio)
                del p_data['HA'][day]['ECG']
                #HA_df['BVP'] = signal.resample(p_data['HA'][day]['BVP']['BVP'], resample_ratio)

                HA_df['EDA'] = signal.resample(p_data['HA'][day]['EDA']['EDA'], resample_ratio)
                del p_data['HA'][day]['EDA']
                HA_df['TEMP'] = signal.resample(p_data['HA'][day]['TEMP']['Temp'], resample_ratio)
                del p_data['HA'][day]['TEMP']
                HA_df['BR'] = signal.resample(p_data['HA'][day]['BR']['breathing_rate'], resample_ratio)
                del p_data['HA'][day]['BR']
                HA_df['ACCx_hx'] = signal.resample(p_data['HA'][day]['ACC_hx']['Acceleration_X'], resample_ratio)
                del p_data['HA'][day]['ACC_hx']['Acceleration_X']
                HA_df['ACCy_hx'] = signal.resample(p_data['HA'][day]['ACC_hx']['Acceleration_Y'], resample_ratio)
                del p_data['HA'][day]['ACC_hx']['Acceleration_Y']
                HA_df['ACCz_hx'] = signal.resample(p_data['HA'][day]['ACC_hx']['Acceleration_Z'], resample_ratio)
                del p_data['HA'][day]['ACC_hx']['Acceleration_Z']
                HA_df['ACCx_e4'] = signal.resample(p_data['HA'][day]['ACC_e4']['Acc_X'], resample_ratio)
                del p_data['HA'][day]['ACC_e4']['Acc_X']
                HA_df['ACCy_e4'] = signal.resample(p_data['HA'][day]['ACC_e4']['Acc_Y'], resample_ratio)
                del p_data['HA'][day]['ACC_e4']['Acc_Y']
                HA_df['ACCz_e4'] = signal.resample(p_data['HA'][day]['ACC_e4']['Acc_Z'], resample_ratio)
                del p_data['HA'][day]['ACC_e4']['Acc_Z']
                HA_df['rot_label'] = rot_anx_dict['HA']* np.ones(resample_ratio) 
                HA_df['calib_label'] = np.zeros(resample_ratio)

                for column in columns_list:
                    if column == 'daily_check_in_date':
                        A = np.chararray(resample_ratio,itemsize=15,unicode=True)
                        A[:] = ((redcap_df.loc[redcap_df['participant'] == p]).iloc[day][column])
                        HA_df[column] = A
                        
                    else:
                        HA_df[column] = np.ones(resample_ratio)* (redcap_df.loc[redcap_df['participant'] == p]).iloc[day][column] 
                        
                print('Done with day ', day, 'date: ',(redcap_df.loc[redcap_df['participant'] == p]).iloc[day]['daily_check_in_date'],' of HA rotation for participant ', p)
                p_df = pd.concat([p_df, HA_df], ignore_index=True, sort=False)
                #print('display HA_df after HA_df')
                #display(HA_df)
                

        
        calib_df = p_data['calib']['BVP'].copy()
        del p_data['calib']['BVP']
        resample_ratio = round(len(calib_df))
        
        calib_df['ECG'] = signal.resample(p_data['calib']['ECG']['ECG'], resample_ratio)
        del p_data['calib']['ECG']
        #calib_df['BVP'] = signal.resample(p_data['calib']['BVP']['BVP'], resample_ratio)
        calib_df['EDA'] = signal.resample(p_data['calib']['EDA']['EDA'], resample_ratio)
        del p_data['calib']['EDA']
        calib_df['TEMP'] = signal.resample(p_data['calib']['TEMP']['Temp'], resample_ratio)
        del p_data['calib']['TEMP']
        calib_df['BR'] = signal.resample(p_data['calib']['BR']['breathing_rate'], resample_ratio)
        del p_data['calib']['BR']
        calib_df['ACCx_hx'] = signal.resample(p_data['calib']['ACC_hx']['Acceleration_X'], resample_ratio)
        del p_data['calib']['ACC_hx']['Acceleration_X']
        calib_df['ACCy_hx'] = signal.resample(p_data['calib']['ACC_hx']['Acceleration_Y'], resample_ratio)
        del p_data['calib']['ACC_hx']['Acceleration_Y']
        calib_df['ACCz_hx'] = signal.resample(p_data['calib']['ACC_hx']['Acceleration_Z'], resample_ratio)
        del p_data['calib']['ACC_hx']['Acceleration_Z']
        calib_df['ACCx_e4'] = signal.resample(p_data['calib']['ACC_e4']['Acc_X'], resample_ratio)
        del p_data['calib']['ACC_e4']['Acc_X']
        calib_df['ACCy_e4'] = signal.resample(p_data['calib']['ACC_e4']['Acc_Y'], resample_ratio)
        del p_data['calib']['ACC_e4']['Acc_Y']
        calib_df['ACCz_e4'] = signal.resample(p_data['calib']['ACC_e4']['Acc_Z'], resample_ratio)
        del p_data['calib']['ACC_e4']['Acc_Z']
        calib_df['rot_label'] = rot_anx_dict['calibration']* np.ones(resample_ratio)
        calib_df['calib_label'] = np.ones(resample_ratio) #complete later
        redcap_dfcalib = redcap_df # a cheat for now since i don't have redcap data for calibration
        columns_list = redcap_dfcalib.columns
        ############################################
        # redcap for calibration doesn't exist yet #
        ############################################

        for column in columns_list:
            #print(column)
            #print((redcap_dfcalib.loc[redcap_dfcalib['participant'] == p]).iloc[0][column]) # this is incorrect for now, replace with calibration data later
            if column == 'daily_check_in_date':
                A = np.chararray(resample_ratio,itemsize=15,unicode=True)
                A[:] = ((redcap_dfcalib.loc[redcap_dfcalib['participant'] == p]).iloc[0][column]) # this is incorrect for now, replace with calibration data later
                calib_df[column] = A
            else:
                calib_df[column] = np.ones(resample_ratio)* (redcap_dfcalib.loc[redcap_dfcalib['participant'] == p]).iloc[0][column] # this is incorrect for now, replace with calibration data later


        p_df = pd.concat([p_df, calib_df], ignore_index=True, sort=False)
        #print('display calib_df after calib')
        #display(calib_df)
        #display(calib_df)



        # save the data
        print('Pickling processed data for participant '+str(p)+ ' ...')
        with open(radwear_path +'p_'+str(p)+'.pkl', 'wb') as f:
            pickle.dump(p_df, f)
        print('data pickled...')
    else:
        print('processed data for participant '+str(p)+ ' is available.')
        #load the data
        #with open(radwear_path +'p_'+str(p)+'.pkl', 'rb') as f:
        #    p_df = pickle.load(f)

pickle file for participant 16 is available.
participant 16 data loaded.
-----------------------
Participant:  16
-----------------------
p_data.keys():  dict_keys(['calib', 'LA', 'HA'])
-----------------------
la days:  0
ha days:  9
-----------------------
Done with day  0 date:  2023-08-28  of LA rotation for participant  16
Done with day  1 date:  2023-08-29  of LA rotation for participant  16
Done with day  2 date:  2023-08-30  of LA rotation for participant  16
Done with day  3 date:  2023-08-31  of LA rotation for participant  16
Done with day  4 date:  2023-09-01  of LA rotation for participant  16
Done with day  5 date:  2023-09-05  of LA rotation for participant  16
Done with day  6 date:  2023-09-06  of LA rotation for participant  16
Done with day  7 date:  2023-09-07  of LA rotation for participant  16
Done with day  8 date:  2023-09-08  of LA rotation for participant  16
Pickling processed data for participant 16 ...
data pickled...
pickle file for participant 17 is avail

In [ ]:
display(HA_df.columns)
display(LA_df.columns)
display(calib_df.columns)

# Create features

# Run the correlation 

In [ ]:
p = 7





for p in list_of_participants:
    p_path = radwear_path + 'Participant '+str(p)
    p_df = pd.DataFrame()
    
    #check if file exist
    a = 'available.' if os.path.isfile(p_path + '/p_'+str(p)+'.pkl') else ' not available.'
    print('processed data for participant '+str(p)+ ' is '+ a)
    print('Loading...')
    with open(p_path +  '/p_'+str(p)+'.pkl', 'rb') as f:
        p_data = pickle.load(f)
    print ('participant '+str(p)+ ' data loaded.')
    print('-----------------------') 
    
    #all_p[p] = p_data ## this takes too much memory so i will just load each p when needed
    print('participant ',p)
    
    if  not (os.path.isfile(radwear_path +'p_'+str(p)+'.pkl')):
        print('processing participant '+str(p)+ ' data...')        

In [ ]:
print(radwear_path)
os.listdir(radwear_path)

In [ ]:
p = 7
#p = 18
p_df = [] 
p_df = pd.read_pickle(radwear_path +'p_'+str(p)+'.pkl')
display(p_data)

sys.getsizeof

# 